# Coursework 1: Finite Fields

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 1.1 Integers (30%)

### 1.1.1 Check a prime number

Write a function `isprime` which checks whether a positive integer is a prime or not. Test your function using at least five different numbers (some primes some not) less than 100. 

In [ ]:
function isprime(num)
    if num == 0 || num == 1
        return false
    end
    for i in 2:num/2
        if mod(num,i) == 0
            return false
        end
    end
    return true
end
println(isprime(13))
println(isprime(48))
println(isprime(53))
println(isprime(97))
println(isprime(73))
println(isprime(63))
println(isprime(3))

### 1.1.2 Extended Euclidean algorithm for integers

1. Get familiar with the function `gcdx` that comes with Julia. 
2. Mimic the input and output style of `gcdx` and write your own function `my_gcdx` for the same functionality.
3. Test your function using at least 5 different instances. 

### 1.1.3 Modular arithmetic 

Let $p=811$, $x=3$, and $n=789$. Compute $x^n \equiv a ~\text{mod}~p$. Show the necessary steps for the computations. 

## 1.2 Polynomials (50%)

Let $p$ be a prime number. Consider the polynomial ring $\mathbb{F}_p[x]$. 

### 1.2.1 Polynomial division

Design and write a function `polydiv' which calculates the quotient $q(x)$ and the remainder $r(x)$ from $f(x) / g(x)$ so that 
1. $f(x) = q(x) g(x) + r(x)$ where $\text{deg}(r(x)) < \text{deg}(g(x))$; 
2. When $r(x) \ne 0$, $r(x)$ is a monic polynomial; 
3. Clear documentation to allow other people to know how to input $f(x)$ and $g(x)$. 
4. Run 5 different tests with simple cases. For example, $p=2$ and $p=3$, $\text{deg}(f) = 3$ and $\text{deg}(f) = 4$. 

In [1]:
function getDegree(p)
    degreeCandidate = 0
    for i in 1:(size(p)[2])
       if (p[i] != 0)
        degreeCandidate = i - 1
       end 
    end
    degree = degreeCandidate
    return degree
end

function rightShift(p, shiftAmount)
    pInner = copy(p)
    for q in 1:shiftAmount
        for i in (size(p)[2]):-1:2
            pInner[i] = pInner[i-1]
        end
        pInner[1] = 0
    end
    return pInner
end

function polydiv(f,g)
    q = Matrix{Float64}(undef, 1, size(f)[2])
    fill!(q, 0.)
    gTemp = rightShift(g, getDegree(f)-getDegree(g))
    while(getDegree(f) >= getDegree(g))
        gTemp = rightShift(g, getDegree(f)-getDegree(g))
        q[(getDegree(f)-getDegree(g))+1] = f[(getDegree(f))+1] / gTemp[(getDegree(gTemp))+1]
        gTemp = gTemp*q[(getDegree(f)-getDegree(g)) + 1]
        f = f - gTemp
        #println(f)
    end
    r = f
    return (q,r)
end

polydiv([7.0 5 1 0 0 0], [1.0 1 0 0 0 0])

([4.0 1.0 … 0.0 0.0], [3.0 0.0 … 0.0 0.0])

### 1.2.2 Extended Euclidean algorithm for polynomials

Design and write a function `gcdx_poly` to compute the GCD of $f(x)$ and $g(x)$, and their Bezout coefficients. Run 5 different tests to demonstrate the correctness of your function.

### 1.2.3 Irreducible polynomial

Design and write a function `is_irreducible` to check whether $f(x) \in \mathbb{F}_p[x]$ is irreducible or not. Run 5 tests to demonstrate the correctness of your function. 

## 1.3 Primitive element (20%)

### 1.3.1 Find primitive elements 

Consider the finite field $\mathbb{F}_p$. Design and write a function `find_primitive` to find all primitive elements in $\mathbb{F}_p$. Test your function with a prime number $20 < p < 50$. 

### 1.3.2 Calculate primitive elements

Now consider the finite field $\mathbb{F}_{32}$. Without programming, can you find all the primitive elements in $\mathbb{F}_{32}$? If so, explain your approach and its rationality. 

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.